# Assignment Three
## Task A

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid

import matplotlib.pyplot as plt

In [2]:
transform = transforms.ToTensor()

train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

In [3]:
train_loader = DataLoader(train_data, batch_size=100, shuffle=True)
test_loader = DataLoader(test_data, batch_size=500, shuffle=False)

In [4]:
class CNNmodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64*7*7, 10)
        #self.dense = nn.Linear(64 * 7 * 14, 10)

    def forward(self, x):
        #x = F.relu(self.conv1(x))
        x = (self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        #x = F.relu(self.conv2(x))
        x = (self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 64*7*7)
        x = self.fc1(x)
        
        return torch.log_softmax(x, dim=1)


In [5]:
model = CNNmodel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
import time
start = time.time()

epochs = 4

train_correct = []

for i in range(epochs):
    
    trn_corr = 0
    
    for b, (x_train, y_train) in enumerate(train_loader):
        
        b+=1
        
        y_pred = model(x_train)
        loss = criterion(y_pred, y_train)
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if b%200==0:
            print(f'Epoch: {i} Loss: {loss} Accuracy: {trn_corr.item()*100/(b*100):7.3f}')
            
    train_correct.append(trn_corr)
        
full_time = time.time() - start
print(f'Training took {(full_time/60):2f} minutes')
    

Epoch: 0 Loss: 0.12549404799938202 Accuracy:  88.370
Epoch: 0 Loss: 0.11473812162876129 Accuracy:  92.243
Epoch: 0 Loss: 0.029618285596370697 Accuracy:  93.888
Epoch: 1 Loss: 0.1066390722990036 Accuracy:  97.790
Epoch: 1 Loss: 0.16070909798145294 Accuracy:  97.983
Epoch: 1 Loss: 0.02428649365901947 Accuracy:  98.018
Epoch: 2 Loss: 0.08039876073598862 Accuracy:  98.500
Epoch: 2 Loss: 0.04561030864715576 Accuracy:  98.430
Epoch: 2 Loss: 0.021818120032548904 Accuracy:  98.427
Epoch: 3 Loss: 0.16522586345672607 Accuracy:  98.775
Epoch: 3 Loss: 0.04233289137482643 Accuracy:  98.787
Epoch: 3 Loss: 0.028312232345342636 Accuracy:  98.717
Training took 1.388357 minutes
